In [ ]:
from scipy.stats import ttest_ind
import helper 
import xarray as xr 
import numpy as np 
import cmaps as cmap 
import matplotlib.pyplot as plt
import scipy 
import warnings
import matplotlib.patheffects as pe
from matplotlib.colors import Normalize
from matplotlib import rcParams
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.patches import Rectangle
import matplotlib.patheffects as pe
warnings.filterwarnings("ignore")

dataset1 = r"C:\Users\deela\Downloads\tc_radar_v3m_1997_2019_xy_rel_swath_ships.nc"
dataset2 = r"C:\Users\deela\Downloads\tc_radar_v3m_2020_2024_xy_rel_swath_ships.nc"
dataset = xr.open_mfdataset([dataset1,dataset2], concat_dim='num_cases', combine='nested')

In [ ]:
print(dataset, list(dataset.variables))

In [ ]:
data = dataset['swath_wind_speed']
print(data)

In [ ]:
print(data.longitude)

In [ ]:
x = data.longitude.values * 2
xCenter = np.nanmean(x)
y = data.latitude.values * 2
yCenter = np.nanmean(y)

X, Y = np.meshgrid(x, y, indexing = 'ij')
r = np.sqrt((X - xCenter)**2 + (Y - yCenter)**2)
print(r)


In [ ]:
dr = 2  # grid-unit bin width (adjust)
rmax = r.max()
bins = np.arange(0, rmax + dr, dr)
r_labels = (bins[:-1] + bins[1:]) / 2

# Flatten spatial dims into one
stacked = data.stack(points=('latitude', 'longitude'))
r_flat  = xr.DataArray(r.ravel(), dims=('points',))

# Group by radius bins
radial_profile = (
    stacked.groupby_bins(r_flat, bins=bins, labels=r_labels)
           .mean(dim='points')
)

# Result: dims = (num_cases, height, r_da_bins)
print(radial_profile)

In [ ]:
radial_profile = radial_profile.sel(group_bins = slice(0, 40))
normProfile = (radial_profile - radial_profile.min(['height', 'group_bins'])) / (radial_profile.max(['height', 'group_bins']) - radial_profile.min(['height', 'group_bins']))
rmax = normProfile.fillna(0).idxmax('group_bins')
r_3d = normProfile['group_bins'].broadcast_like(normProfile)
r_norm = r_3d / rmax
normProfile = normProfile.assign_coords(group_bins=r_norm)
print(normProfile)
vmax = dataset['vmax_ships'].sel(ships_lag_times = 0)
correlation = xr.corr(normProfile, vmax, dim = 'num_cases')

In [ ]:
a = normProfile.compute().idxmax('group_bins')
print(a)

In [ ]:
print(correlation)

In [ ]:
dataToPlot = correlation.values

In [ ]:
fig = plt.figure(figsize=(12, 9))
ax = plt.axes()
ax.grid(linewidth = 0.5, color = 'black', alpha = 0.5, linestyle = '--', zorder = 10)
ax.set_frame_on(False)
c = ax.pcolormesh(correlation.group_bins, correlation.height, dataToPlot, vmin = -1, vmax = 1, cmap = cmap.tempAnoms3())
ax.set_title('Correlation between Normalized Wind and VMax\nAll TC-RADAR Swaths', fontweight = 'bold', loc = 'left')
ax.set_title('Deelan Jariwala', loc = 'right')
cbar = plt.colorbar(c, orientation='vertical', aspect=50, pad=0.02)
plt.savefig(r"C:\Users\deela\Downloads\tdrWindCorr.png", dpi = 400, bbox_inches = 'tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=(12, 9))
ax = plt.axes()
ax.grid(linewidth = 0.5, color = 'black', alpha = 0.5, linestyle = '--', zorder = 10)
ax.set_frame_on(False)
c = ax.pcolormesh(correlation.group_bins, correlation.height, normProfile.mean('num_cases'), vmin = 0, vmax = 1, cmap = cmap.tempAnoms3())
ax.set_title('Correlation between Normalized Wind and VMax\nAll TC-RADAR Swaths', fontweight = 'bold', loc = 'left')
ax.set_title('Deelan Jariwala', loc = 'right')
cbar = plt.colorbar(c, orientation='vertical', aspect=50, pad=0.02)
plt.savefig(r"C:\Users\deela\Downloads\tdrWindCorr.png", dpi = 400, bbox_inches = 'tight')
plt.show()